# Extension of GSCAN results to LIBD Data eQTL maps: baseline, interaction, and stratified 2df

**Author:** Jesse Marks <br>
**GitHub Issue:** [LIBD NAc eQTL Analysis #122](https://github.com/RTIInternational/bioinformatics/issues/122)<br>
**Results**: `~/Desktop/Projects/LIBD/gscan_lookup/20190823/20190823_libd_eqtl_analysis_gscan_tables1-4_snp_lookup_results_v02.xlsx`

This notebook documents the steps taken to perform a SNP lookup of the GW significant SNPs from GSCAN in our eQTL analysis. In particular, Bryan Quach performed the eQTL analysis on the human nucleus accumbens data [(NAc)](https://en.wikipedia.org/wiki/Nucleus_accumbens) provided by Lieber Institute for Brain Development (LIBD).


For top GSCAN results, see the supplemental Tables S1-S4 here:<br>
`\\rtpnfil02\dhancock\Analysis\GSCAN\PublishedVersion\GSCAN_SupplTables.xlsx`.

<br>

The GSCAN phenotypes of interest to us include: 

1) **Age of smoking initiation (AI)** - supplemental table 1

2) **Cigarettes per day (CPD)** - supplemental table 2

3) **Smoking cessation (SC)** - supplemental table 3

4) **smoking initiation (SI)** - supplemental table 4


<br>
<br>
<br>

___

**Note**: Bryan Quach pointed me to the data on his master EC2 instance (100.24.91.78 bquach):
* **Baseline Results**: `/shared/data/libd/eqtl/data/results/baseline/*/libd_nac_eqtl_chr_all_counts_based_cis_qtl_updated_table_mac_lt_3.txt` 

* **Interaction Results**: 
`/shared/data/libd/eqtl/data/results/interaction/*/libd_nac_eqtl_chr_all_counts_based_cis_qtl_updated_table_mac_lt_3.txt`

* **Stratified 2df Results**:
`/shared/data/libd/eqtl/data/results/stratified_smoking_2df/*/libd_nac_eqtl_chr_all_counts_based_cis_qtl_updated_table_mac_lt_3.txt`

<br>

* **SNP ID to location map**:
`/shared/data/libd/eqtl/data/snp_locations/`

## Download Data

Create the directory structure.  Also, create a list of SNPs for the lookup from the GSCAN Excel sheet. Copy the SNPs from each of the four Excel sheets (supplemental tables) into separate files and then combine them (no duplicates).

**Note**: We do not have to download the data from S3 for this case because the data was already residing on Bryan Q.'s EC2 instance.

In [ ]:
## EC2 ##

# Create directory 
dir=/shared/jmarks/libd/snp_lookup/20190821
mkdir -p $dir/{baseline,interaction,stratified}/{cross,ea,aa}/lookup_results


# populate these files with the SNPs from respective Excel sheets

touch age_of_initiation.tsv  cig_per_day.tsv  smoking_cessation.tsv  smoking_initiation.tsv
wc -l * # note these are with headers
"""
   11 age_of_initiation
   56 cig_per_day
   25 smoking_cessation
  379 smoking_initiation
"""

head age_of_initiation.tsv
"""
Chr     Pos     rsID    Reference Allele        Alternate Allele
2       145638766       rs72853300      C       T
2       225353649       rs12611472      T       C
2       63622309        rs7559982       T       A
3       85699040        rs11915747      C       G
4       140908755       rs13136239      G       A
"""

# combine SNPs into one file, make sure SNPs are not listed twice
head -1 age_of_initiation.tsv > combined_snp_list.tsv
for file in age_of_initiation.tsv cig_per_day.tsv smoking_cessation.tsv smoking_initiation.tsv; do
    tail -n +2 $file >> combined_snp_list.tsv
done

# filter so that there are no duplicated SNPs (no header either)
tail -n +2 combined_snp_list.tsv | sort -u > combined_snp_list_filtered.tsv

# convert to 1000g_p3 format
awk '{print $3":"$2":"$4":"$5"\t"$1}' combined_snp_list_filtered.tsv > combined_snp_list_filtered_1000g_p3_chr.tsv

``` 
wc -l combined_snp_list_filtered_1000g_p3_chr.tsv
462 combined_snp_list_filtered_1000g_p3_chr.tsv

 head combined_snp_list_filtered_1000g_p3_chr.tsv
rs7920501:10043159:T:A  10
rs7901883:103186838:G:A 10
rs11594623:103960351:T:C        10
rs11191269:104120522:C:G        10
rs28408682:104403310:A:G        10
rs12244388:104640052:G:A        10
rs111842178:104852121:A:G       10
rs34970111:106078937:C:T        10
rs9787523:106460460:T:C 10
rs11192347:106929313:G:A        10
```

## Recapture Missing SNPs
Some of the SNPs in the eQTL analysis results were of a different format: 
```
	 rs12142098:59009279:T:C	1:100016494:T:TAATAA	rs2180184	GA000234	1:156556241:C:<INS:ME:SVA>:156557143
```

I will try to capture those SNPs by including in my input three different formats of the same SNP. For example:
```
rs7920501
rs7920501:10043159:T:A
10:rs7920501:T:A
```
This is three different formats for the same SNP. So, our list of SNPs will be $462 \times 3 = 1,386$

In [ ]:
### python3 ###

infile = "combined_snp_list_filtered.tsv"
outfile = "combined_snp_list_filtered_all_formats.tsv"

with open(infile) as inF, open(outfile, "w") as outF:
    line = inF.readline()
    while line:
        sl = line.split()
        chrom = sl[0]
        pos = sl[1]
        rsid = sl[2]
        a1 = sl[3]
        a2 = sl[4]

        id1 = "{}:{}:{}:{}".format(rsid,pos,a1,a2)
        id2 = "{}:{}:{}:{}".format(chrom,pos,a1,a2)

        line1 = "{}\t{}\n".format(rsid, chrom) # rsid and chromosome
        line2 = "{}\t{}\n".format(id1, chrom)  # rsid:pos:a1:a2 and chromosome
        line3 = "{}\t{}\n".format(id2, chrom)  # chrom:pos:a1:a2 and chromosome

        outF.write(line1)
        outF.write(line2)
        outF.write(line3)

        line = inF.readline()

## SNP lookup
I need to create a dictionary of the GSCAN SNPs and then see if each SNP in the meta-analysis is in the dictionary. I think this makes more sense rather than vice-versa; in particular, creating a dictionary for each SNP in the meta-analysis and then searching to see if the GSCAN SNPs are in the dictionary. This latter strategy would require a large amount of memory to create the Python dictionary. The former strategy makes more sense when comsidering computation expense. 

Also note that there are some SNPs in the meta-analysis which have the format of `chr:position:a1:a2` instead of `rsid:position:a1:a2`. I think the reason is that these SNPs of the former format did not have an associated rsID available. If a GSCAN SNP is not found in the lookup, then we need to output the SNPs that were not found and deal with those later. It might be the case that we have to convert them from `rsid:position:a1:a2` format to `chr:position:a1:a2` and then perform the search again with just these SNPs.

### Baseline

#### AA

#### EA

#### Cross

In [ ]:
### Python3 ###
"""
*SNP lookup*

    Make sure the IDs are of the same format for the snp-list
    and the IDs in the  eQTL analysis results. e.g. 1000g_p3 or rsID only
"""
#import gzip

################################################################################
date = "20190821" # date the results were generated (in results files name)
ancestry = "aa"
study = "baseline"
file_name = "libd_nac_eqtl_chr_all_counts_based_cis_qtl_updated_table_mac_lt_3.txt"

## dict to hold gscan snps and the number of times they were found.
## we can tell which SNPs did not show up in any of the eQTL files
gscan_dict =  {}

# directory contaiting combined_snp_list_filtered_1000g_p3_chr.tsv
base_dir = "/shared/jmarks/libd/snp_lookup/20190821/{}/{}".format(study, ancestry)
snp_list = "/shared/jmarks/libd/snp_lookup/20190821/combined_snp_list_filtered_1000g_p3_chr.tsv".format(base_dir)
results = "{}/{}".format(base_dir, file_name)

out_file = "{}/lookup_results/{}_libd_{}_{}_nac_eqtl_analysis_snp_lookup_results.txt".format(base_dir, date, study, ancestry)
not_found = "{}/lookup_results/{}_libd_{}_{}_nac_eqtl_analysis_snp_lookup_results_not_found.txt".format(base_dir, date, study, ancestry)
################################################################################

with open(results) as qtlF, open(snp_list) as gscanF, open(out_file, "w") as outF:
    gscan_line = gscanF.readline()
    qtl_head = qtlF.readline()
    qtl_line = qtlF.readline()

    print(qtl_head)
    #outF.write(qtl_head)

    ## create a dictionary containing the gscan snps (just 1000G_p3 portion)
    if len(gscan_dict) == 0:
        while gscan_line:
            key = gscan_line.split()[0] # SNP id
            value = gscan_line.split()[1] #chromosome
            gscan_dict[key] = [0, value]
            gscan_line = gscanF.readline()

    while qtl_line:
        qtl_id = qtl_line.split()[0] # the 1000g_p3 ID in the eqtl-analysis
        if qtl_id in gscan_dict:
            gscan_dict[qtl_id][0] += 1
            outF.write(qtl_line)
        qtl_line = qtlF.readline()

with open(not_found, "w") as notF:
notF.write("id\tchr\n")
for key, value in gscan_dict.items():
    if value[0]==0:
        notF.write(key + "\t" +  value[1] + "\n")


### Interaction

#### AA

#### EA

#### Cross

### Stratified 2df

#### AA

#### EA

#### Cross

# Missing SNPs

In [ ]:
### local ###
cd ~/Desktop/Projects/LIBD/gscan_lookup/20190823/

for study in {baseline,stratified,interaction}; do
    for an in {aa,ea,cross}; do
        tail -n +2 ${study}/${an}/lookup_results/20190823_libd_${study}_${an}_nac_eqtl_analysis_snp_lookup_results_not_found.txt | \
            cut -f2,3 | uniq -c | awk '$1==3 {print $3,$2}' > ${study}/${an}/lookup_results/snps_not_found.txt
    done
done